In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [19]:
df = pd.read_csv('data_onevisa_postprocess.csv').set_index('sample_id')
df.shape

/var/folders/f0/dt7lkxv100q68c369j1bzgk00000gn/T/ipykernel_22258/3320170570.py:1: DtypeWarning: Columns (247,273,334,342,344,345,358,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,405,407,412,416,431,432,433,434,435,436,437,438,439,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,488,489,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,578,579,580,581,582) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data_onevisa_postprocess.csv').set_index('sample_id')


(6742, 603)

In [20]:
summary = pd.DataFrame({
        "number_unique": df.nunique(),
        "number_nan": df.isna().sum(),
        "dtype_df": df.dtypes
    })

In [21]:
df['generated'] = df['generated'].apply(lambda x: x.split(',')[1])

In [22]:
df['generated_date'] = pd.to_datetime(df['generated'], dayfirst=True)


In [23]:
list_columns_remove = summary[(summary.number_unique>df.shape[0]*0.8) & (summary.dtype_df=='object')].sort_values(by='number_unique').index.tolist()
list_columns_remove_f = [i for i in list_columns_remove if 'date' not in i]

In [24]:
df_f = df.drop(columns = list_columns_remove_f)

In [25]:
df_f.shape

(6742, 588)

In [26]:
date_cols = [c for c in df.columns if 'date' in c.lower()]

# 2) Compute their missing‐value percentages
miss_pct = df_f[date_cols].isna().mean()

# 3) Drop those over 80% missing
to_drop = miss_pct[miss_pct > 0.8].index
df_f = df_f.drop(columns=to_drop)
print(df_f.shape)

(6742, 545)


In [27]:
df_f = df_f[df_f['date_of_issue_nic']!='Place of birth']
df_f = df_f[df_f['date_of_expiry_nic']!='Place of birth']

df_f = df_f.drop(columns = ['date_from_e_pd','date_to_e_pd','date_from_cv',
                            'date_to_cv','date_from_ehd','date_to_ehd','date_from_e_scd',
                            'date_to_e_scd'])

In [28]:
date_cols = [
    c for c in df_f.columns
    if 'date' in c.lower() and c != 'generated_date']

for col in date_cols:
    print(col)
    # convert the column to datetime if it isn’t already
    df_f[col] = pd.to_datetime(df_f[col], dayfirst=True)
    # replace with difference in days (you can keep the Timedelta if you prefer)
    df_f[col] = (df_f['generated_date'] - df_f[col]).dt.days

date_of_birth_pa_pd
date_of_issue_pa_pd
date_of_expiry_pa_pd
date_of_issue_nic
date_of_expiry_nic
date_of_birth_cdc
date_of_birth_ofm_fmd
date_from_hi_oshc
date_to_hi_oshc
date_of_test_l_etd


In [29]:
today = pd.Timestamp.today().normalize()
df_f['days_from_today'] = (today - df_f['generated_date']).dt.days

df_f = df_f.drop(columns= ['generated_date'])

In [39]:
df_f = df_f[df_f.in_australia_x!='Character declarations']

In [40]:
df_f.shape

(5340, 537)

In [41]:
summary = pd.DataFrame({
        "number_unique": df_f.nunique(),
        "number_nan": df_f.isna().sum(),
        "dtype_df": df_f.dtypes
    })

In [42]:
df_f.type_case.value_counts()

type_case
granted    5019
refused     321
Name: count, dtype: int64

In [43]:
summary[summary.index.isin([i for i in df_f.columns if 'australia' in i])]

,number_unique,number_nan,dtype_df
australia_ac_coe,2,0,object
is_this_course_a_continuation_of_the_applicant_s_studies_or_training_outside_australia_coed,2,148,object
does_the_applicant_hold_evidence_of_their_intended_study_in_australia_other_than_a_coe_coed_oeois,2,148,object
is_the_applicant_applying_for_this_visa_due_to_the_closure_of_their_australian_education_provider_coed_coep,2,157,object
commonwealth_government_of_australia_coed_cfs,1,157,object
has_this_applicant_undertaken_a_health_examination_for_an_australian_visa_in_the_last_12_months_nic_he,2,468,object
has_this_applicant_previously_travelled_to_australia_or_previously_applied_for_a_visa_aiq_ptta,3,1547,object
does_the_applicant_have_any_parents_or_siblings_in_or_outside_australia_ofm,2,0,object
value_in_australian_dollars,1220,7,float64
australia_e_eh,2,0,object


In [44]:
df_f[df_f.stay_in_australia_itehohcf=='Character declarations']

,current_location_ac_cl,legal_status_ac_cl,covid_19_ac_at,australia_ac_coe,is_this_course_a_continuation_of_the_applicant_s_studies_or_training_outside_australia_coed,does_the_applicant_hold_evidence_of_their_intended_study_in_australia_other_than_a_coe_coed_oeois,note_coed_oeois,education_sector_coed_es,is_the_applicant_applying_for_this_visa_due_to_the_closure_of_their_australian_education_provider_coed_coep,commonwealth_government_of_australia_coed_cfs,information_pa,given_names_pa_pd,sex_pa_pd,date_of_birth_pa_pd,country_of_passport_pa_pd,nationality_of_passport_holder_pa_pd,date_of_issue_pa_pd,date_of_expiry_pa_pd,authority_pa_pd,does_this_applicant_have_a_national_identity_card_pa_nic,given_names_nic,country_of_issue_nic,note_nic,date_of_issue_nic,date_of_expiry_nic,town_city_nic_pob,state_province_nic_pob,country_of_birth_nic_pob,relationship_status_nic_rs,is_this_applicant_currently_or_have_they_ever_been_known_by_any_other_names_nic_on/s,is_this_applicant_a_citizen_of_the_selected_country_of_passport_x_nic_c,is_this_applicant_a_citizen_of_any_other_country_nic_c,does_this_applicant_have_other_current_passports_nic_op,does_this_applicant_have_other_identity_documents_nic_oid,has_this_applicant_undertaken_a_health_examination_for_an_australian_visa_in_the_last_12_months_nic_he,given_names_cdc,sex_cdc,date_of_birth_cdc,country_of_birth_cdc,country_of_passport_cdc,correct_cdc,has_this_applicant_previously_travelled_to_australia_or_previously_applied_for_a_visa_aiq_ptta,are_there_any_accompanying_members_of_the_family_unit_included_in_this_application_amotfu,usual_country_of_residence_cd_cor,the_closest_office_to_the_applicant_s_current_location_cd_do,office_cd_do,country_cd_ra,suburb_town_cd_ra,state_or_province_cd_ra,postal_code_cd_ra,is_the_postal_address_the_same_as_the_residential_address_cd_pa,email_address_cd_ea,does_the_applicant_authorise_another_person_to_receive_written_correspondence_on_their_behalf_ar,family_name_ar_ea,given_names_ar_ea,organisation_ar_ea,country_ar_pa,address_ar_pa,suburb_town_ar_pa,state_or_province_ar_pa,postal_code_ar_pa,business_phone_ar_ctn,mobile_cell_phone_ar_ctn,all_correspondence_including_notification_of_the_outcome_of_the_application_will_be_sent_to_ar_ec,email_address_ar_ec,accompanying_member_of_the_family_unit_in_this_application_nmotfu,does_the_applicant_have_any_parents_or_siblings_in_or_outside_australia_ofm,applicant_ofm_rtpa,given_names_ofm_fmd,sex_ofm_fmd,date_of_birth_ofm_fmd,usual_country_of_residence_ofm_ucor,also_be_requested_ffs,will_the_funds_be_provided_by_an_individual_other_than_the_applicant_ffs_fsfai,relationship_to_the_applicant_ffs_fsfai,other_relationship_ffs_fsfai,funding_type_ffs_fsfai,value_in_australian_dollars,does_the_applicant_have_overseas_student_health_cover_x_hi_oshc,was_the_overseas_student_health_cover_x_organised_by_the_applicant_s_education_provider_hi_oshc,name_of_health_insurer_hi_oshc,date_from_hi_oshc,date_to_hi_oshc,completed_e_hlos,course_name_e_hlos,institution_name_e_hlos,country_of_institution_e_hlos,australia_e_eh,has_the_applicant_previously_studied_in_australia_e_eh,employment_status_e_ehd,is_this_the_applicant_s_current_employment_situation_e_ehd,industry_type_e_e/bd,country_e_oa,suburb_town_e_oa,state_or_province_e_oa,postal_code_e_oa,family_name_e_cpd,given_names_e_cpd,mobile_cell_phone_e_cpd,position_e_pd,has_the_applicant_been_offered_a_job_at_the_completion_of_their_course_e_fe,give_details_of_the_type_of_employment_the_applicant_will_be_seeking_at_the_completion_of_their_course_e_fe,has_the_applicant_undertaken_an_english_language_test_within_the_last_24_months_l_la,main_language_l_ml,or_united_states_of_america_l_siel,requirements_for_a_senior_secondary_certificate_of_education_x_l_siel,australia_l_siel,have_any_of_the_applicants_visited_any_countries_in_the_past_10_years_include_cv,or_any_other_country_vh,country_and_not_complied_with_visa_conditions_or_departed_outside_their_authorised_period_of_stay_vh,further_stay_in_aus

In [45]:
df_f.stay_in_australia_itehohcf.unique()

array([nan, 'No'], dtype=object)

In [47]:
df_f.shape

(5340, 537)

In [46]:
df_f.to_csv('data_onevisa_postprocess_v2.csv', index=True)